# 에라스토테네스의 체 효율적으로 구현하기

In [4]:
import time
import sys
from functools import wraps

# decorator을 사용해서 함수의 실행시간을 측정함
# 어떤 알고리즘이 더 빠른지 벤치마킹 테스트를 할 때 사용함

def multi_elapsed(n):
    def elapsed(f):
        @wraps(f)
        def wrap(*args, **kwargs):
            start_r = time.perf_counter()
            start_p = time.process_time()
            # 함수 실행
            ret = [0]*n
            for i in range(n):
                ret[i] = f(*args, **kwargs)

                if n >= 100:
                    if i % (n//100) == 0:
                        sys.stdout.write(f'\r{f.__name__} : {i // (n//100) + 1}% 진행됨 ')
            end_r = time.perf_counter()
            end_p = time.process_time()
            elapsed_r = end_r - start_r
            elapsed_p = end_p - start_p

            print(f'\n{f.__name__} elapsed: {elapsed_r:.6g}sec (real) / {elapsed_p:.6g}sec (cpu) / {n} repeated')
            return ret
        return wrap
    return elapsed

In [ ]:
# 보통 구현하는 방식 -> n = 백만(1000000) 기준 새 방식에 비해 5배쯤 느림
@multi_elapsed(1000)
def g(n):
    eras = [True]*n
    for i in (2, *range(3, int(n**0.5)+1, 2)):
        if eras[i]:
            for j in range(i*2, n, i):
                eras[j] = False
    return eras

@multi_elapsed(1000)
def h(n):
    # Remark: (1 | 3*i + 1)의 의미는, 3*i 이상의 가장 가까운 짝수 + 1임.
    if n < 5:
        return [i for i in range(n) if i in (2, 3)]
    n = n + (6 - n%6) # n 이상, 가장 가까운 6의 배수
    c = 2 - (n%6 > 1) # n%6이 0, 1이면 2. 그 외엔 1
    sieve = [True] * (n//3)
    for i in range(1, int(n**0.5)//3 + 1):
        if sieve[i]:
            step, square, j = (k := 1 | 3*i + 1) * 2, k * k, k * (k + 4 - 2 * (i & 1))
            sieve[square // 3::step] = [False] * ((n//6 - square//6 - 1) // k + 1)
            sieve[j // 3::step] = [False] * ((n//6 - j//6 - 1) // k + 1)
    return [2, 3] + [1 | 3 * i + 1 for i in range(1, n // 3 - c) if sieve[i]]

g(1000000)
h(1000000)